<h1><center>Laboratorio 9: Los huesos de Hipócrates 🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Matías Villa
- Nombre de alumno 2: Cristóbal Bravo


### **Link de repositorio de GitHub:** `https://github.com/matiasjvd/MDS7202-Laboratorios`

### Indice 

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripcción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

## Temas a tratar

- Creación de clasificadores de imagenes a traves de redes Fully connected y CNN.
- Uso de Dataloaders para la carga de datasets.
- Comparación de Fully Connected y red convolucional.

## Reglas:

- Fecha de entrega: 17/11/2022
- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Creación de modelos de clasificación de imágenes utilizando Pytorch.
- Creación de dataloader y aplicar transformaciones sobre el dataset.
- Comprender la diferencia entre una CNN y una Fully Connected.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones de `Pytorch`, la cual, está enfocada para proyectos de Deep Learning.


# Importamos librerias utiles 😸

Comenzamos importando librerías utiles para la ejecución del laboratorio:

In [1]:
!pip install torch torchvision

In [2]:
import os
import time
from PIL import Image

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import datasets, models
from torchvision import transforms as T

# Identificando los Huesos de Hipócrates🔎

<p align="center">
  <img src="https://i.pinimg.com/originals/d8/58/66/d85866cd1cc3979f94526551addf74b4.gif" width="300">
</p>

Tras el éxito que han tenido proponiendo modelos de machine learning en trabajos anteriores, el famoso medico Hipócrates se ha contactado con ustedes para solicitarles ayuda para automatizar la identificación de radiografías de partes humanas. Para esto, les señala que le gustaría utilizar algoritmos de deep learning producto que Demócrito le señalo que resultan la mejor alternativa para la predicción de imágenes.

En su conversación con el medico usted le comenta que ha tenido algunas clases relacionadas a Deep Learning, por esto, están motivados en abordar el problema utilizando redes Fully Connected y redes convolucionales con Pytorch. Sin embargo, al anunciarle los tipos de redes que conocen, el filósofo les comenta que no había escuchado muy buenos resultados por parte de las CNN, por lo que les pide que comprueben a traves de la métrica de accuracy que tipo de redes es mejor para la tarea de identificación de radiografías. ¿Será cierto lo que dice el filósofo?, Veámoslo en un nuevo capítulo de los Laboratorios de Programación Científica para Ciencia de Datos!

## 1.1 Creación de Lista de Archivos

<p align="center">
  <img src="https://media.tenor.com/BJ-9w-MUVCMAAAAM/tis100-sad.gif" width="300">
</p>


Comience revisando de forma manual cada una de las imágenes que posee la carpeta subida a material docente. Verifique la cantidad de tipos de radiografías que se tienen y la cantidad de imágenes que dispone cada carpeta.

Revisado el contenido de las imágenes, utilice `os.listdir` para crear un `numpy.array` o un `Dataframe` que contenga las imágenes y un label que señale al tipo de radiografía a la que hace referencia la imagen. Para hacer las etiquetas codifique el tipo de imágenes en números que vayan del 0 al total de tipos de radiografías, no utilice strings para codificar las etiquetas.

**Ejemplo de Estructura:**

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-0lax"></th>
    <th class="tg-0lax">image_path</th>
    <th class="tg-0lax">label</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-0lax">0</td>
    <td class="tg-0lax">image1</td>
    <td class="tg-0lax">1</td>
  </tr>
  <tr>
    <td class="tg-0lax">1</td>
    <td class="tg-0lax">image2</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">2</td>
    <td class="tg-0lax">image3</td>
    <td class="tg-0lax">2</td>
  </tr>
  <tr>
    <td class="tg-0lax">3</td>
    <td class="tg-0lax">image4</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">4</td>
    <td class="tg-0lax">image5</td>
    <td class="tg-0lax">4</td>
  </tr>
</tbody>
</table>

In [3]:
folder = "./Medical-MNIST/Medical-MNIST"
imagen,tipo = list(),list()
i = 1
for k,tipo_str in enumerate(os.listdir(folder)):
    ruta = folder+"/"+tipo_str
    for _ in os.listdir(ruta):
        imagen += ["imagen"+str(i)]
        i += 1
        tipo += [k]
df_label = pd.DataFrame({'image path':imagen, "label": tipo})
df_label

,image path,label
0,imagen1,0
1,imagen2,0
2,imagen3,0
3,imagen4,0
4,imagen5,0
...,...,...
58949,imagen58950,5
58950,imagen58951,5
58951,imagen58952,5
58952,imagen58953,5


## 1.2 Creación de Dataset

Tomando en cuenta la estructura de datos desarrollada en el punto 1.1, construya la clase `MedicalDataset()` que cumpla los siguientes puntos:

- [X] Poseer un `__init__` en el que se almacene `estructura` creada en 1.1, la `raiz` de la carpeta y una función que permita transformar el dataset (de esto no se preocupe mucho, ya que solamente debe almacenar una función en el atributo).
- [X] La clase debe ser capaz de entregar la cantidad de elementos a traves de `__len__`.
- [X] Debe poseer el método `__getitem__` que retorne una tupla con la imagen y su correspondiente etiqueta.

In [4]:
enumerate(os.listdir(folder))

In [5]:
class MedicalDataset(Dataset):
    """Esta clase representa el conjunto de los datos médicos, procesados a través de una transformación.

    Atributos
    ---------
    raiz: str
        La ruta de la carpeta donde se encuentran las imágenes
    transform: callable
        Transformación de las imágenes guardadas.
    codificacion: dict[int, str]
        Enumeración de las etiquetas de las imágenes
    estructura: pd.DataFrame
        Frame que asocia una ruta de una imagen a su etiqueta."""
    def __init__(self, raiz: str, transform: callable = lambda x: x):
        """
        Parametros:
        -----------
        raiz: str
            La ruta de la carpeta donde se encuentran las imágenes
        transform: callable, default = función identidad
            Transformación de las imágenes guardadas."""
        self.raiz = raiz
        self.transform = transform
        self.codificacion =  enumerate(os.listdir(raiz))
        self.__construir_estructura()

    def __construir_estructura(self):
        path_label = []
        for label, category in self.codificacion:
            path_category = self.raiz+"/"+category
            path_label += [(path_category+"/"+image, label) for image in os.listdir(path_category)]
        self.estructura = pd.DataFrame(path_label, columns=["image_path", "label"])

    def __getitem__(self, i: int) -> tuple:
        """Retorna una tupla con la i-ésima imagen y su etiqueta correspondiente.

        Parámetros
        ----------
        i: int
            Índice del dato a obtener.

        Retorna
        -------
        image
            Imagen transformada según self.transform.
        label: int
            Etiqueta codificada de la imagen."""
        img_path, label = self.estructura.iloc[i]
        image = self.transform(Image.open(img_path).convert('RGB'))
        return (image, label)

    def __len__(self) -> int:
        """Retorna la cantidad de datos en el dataset."""
        return len(self.estructura)

    def show(self, i: int) -> None:
        """Muestra la i-ésima imagen.

        Parámetros
        ----------
        i: int
            Índice de la imagen a mostrar."""
        image, label = self[i]
        fig = px.imshow(np.array(image).T)
        fig.update_layout(title="Imagen "+str(i)+". Etiqueta: "+self.codificacion[label])
        fig.show()

## 1.3 Prueba del MedicalDataset

Con la clase construida en el punto 1.2, verifique su funcionamiento cargando el dataset y realizando las transformaciones que entrega la función `transform_image`. Compruebe a través de un ejemplo las transformaciones aplicadas en la imagen, comentando la función que cumple `MedicalDataset` y si es posible observar todas las transformaciones aplicadas con la función `transform_image`.

- [X] Probar la clase MedicalDataset y aplicando una transformación de "train"
- [X] Plotear un ejemplo del MedicalDataset.

**Función para transformar las imagenes:**

In [6]:
imgsize = 64
def transform_image(stage=None):
    Tr_img = T.ToTensor()
    if stage == "train":
        Tr_img = T.Compose([T.Resize(size=(imgsize, imgsize)),
                            T.RandomRotation(degrees=(-20, 20)),
                            T.ToTensor()
                           ])
    elif stage == "test" or stage == "val":
        Tr_img = T.Compose([T.Resize(size=(imgsize, imgsize)),
                            T.ToTensor()
                           ])
    return Tr_img

def transform_image(stage = None):
    
    if stage == "train":
        Tr_img = T.Compose([T.Resize(size = (256,256)),
                T.RandomRotation(degrees = (-20,+20)),
                T.ToTensor()])
        
    elif stage == "test" or stage == "val":
        Tr_img = T.Compose([T.Resize(size = (224,224)), T.ToTensor()]) 

    return Tr_img

**Código para obtener un ejemplo:**

In [7]:
dataset = MedicalDataset("Medical-MNIST", transform_image("train"))
indexes = iter(range(len(dataset)))

In [8]:
dataset.show(next(indexes))

PermissionError: [Errno 13] Permission denied: 'Medical-MNIST/Medical-MNIST/AbdomenCT'

In [ ]:
# Prueba del dataset
dataset = MedicalDataset(..., "ruta", transform_image("train"))

In [ ]:
data_test= iter(dataset)
example_image = next(data_test)

# Utilice plotly para plotear un ejemplo
px.imshow(...)

> Comente que realiza la clase construida y las transformaciones aplicadas.

## 1.4 Creación de Clasificadores

<p align="center">
  <img src="https://149695847.v2.pressablecdn.com/wp-content/uploads/2018/01/conv-full-layer.gif" width="300">
</p>

A continuación, deben construir tres clasificadores con los que deberán verificar cuál de las arquitecturas posee un mejor desempeño para la tarea de clasificación de imágenes. Para la construcción considere los siguientes puntos:

- [ ] Señale cual es el objetivo del `forward` en este tipo de redes, sea breve para su explicación.
- [ ] Construir una red Fully Connected para solucionar el problema de clasificación. Para esta parte se le aconseja que rellene el esqueleto dispuesto más abajo y que lleva el nombre de `FCClassifier`, en el deberá rellenar con la dimensión de las capas ocultas y verificar cual será el tamaño de la entrada.
- [ ] Construya una red convolucional **simple** (no más de una capa convolucional) para la tarea de clasificación de imágenes, para esto basen su código en la clase del día `09-11-2022`.
- [ ] Crear una red convolucional más compleja. Para esta parte tienen completa libertad en la construcción de su red, lo único que debe cumplir es que sea convolucional.

**Esqueletos Propuestos:**

In [ ]:
class FCClassifier(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(FCClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(in_channels, ...)
        self.lin2 = nn.Linear(..., num_classes)
        
    def forward(self, x):
        ...

        return x

In [ ]:
class CNNClassifier1(nn.Module):
    ...
    
class CNNClassifier2(nn.Module):
    ...

## 1.5 Separando Datos para el Entrenamiento

<p align="center">
  <img src="https://c.tenor.com/Esn7Jif-ZLQAAAAC/separate-square.gif" width="200">
</p>

Utilizando un Holdout a su gusto, separe los datos en un conjunto de entrenamiento y de testing. Aplique las transformaciones correspondientes usando `transform_image` para cada conjunto de datos y utilice `torch.utils.data.DataLoader` para crear un objeto iterable del dataset.

- [ ] Definir el Holdout a utilizar.
- [ ] Separar los datos en un conjunto de entrenamiento y prueba.
- [ ] Aplicar las transformaciones correspondientes en cada uno de los dataset.
- [ ] Utilizar `DataLoader` de pytorch sobre los dataset.

In [ ]:
# Separar Datos de Entrenamiento
...

In [ ]:
# Uso de torch.utils.data.DataLoader
train_loader = ...
test_loader = ...

## 1.6 Creación de Funciones de Entrenamiento y Evaluación

<p align="center">
  <img src="https://www.researchgate.net/publication/319535615/figure/fig3/AS:536187598065664@1504848493070/A-typical-convolutional-neural-network-CNN-Architecture-for-Medical-Image-Classification.png" width="500">
</p>


Ya construido todas las funciones y clases necesarias llego el momento más importante... probar la red. Para esta sección, ustedes deberán ser capaces de definir los hiperparámetros de la red, definir las funciones de perdida a utilizar, señalar el optimizador a usar y finalmente crear sus funciones para el entrenamiento y prueba. Para realizar esta parte más estructurada, seguir los siguientes puntos de forma secuencial:

- [ ] Especifique los Hiperparámetros de las 3 redes. Para esta parte sea claro de su elección y señale el porqué de sus elecciones (o sea justifique el setting de sus hiperparámetros).
- [ ] Defina los modelos a utilizar, el optimizador que utilizará para el modelo y señale la función de perdida que utilizará.
- [ ] Explique de forma breve la función que cumplen los pasos `Backward` y `Descenso del gradiente` en una red neuronal.
- [ ] Cree una función llamado `train` que entrene a los clasificadores. Para esto, recuerde que estos modelos suelen utilizar un número de épocas, por lo que deberá generar un proceso iterativo de entrenamiento. Es importante que su función imprima las `loss` obtenidas por el modelo en cada época (si gusta puede almacenar estas losses en una lista para luego graficarlas y comparar).
- [ ] Diseñe una función para evaluar el desempeño de las redes. Para evaluar las redes utilice solamente la métrica accuracy (para esto se le recomienda comparar la predicción con el ground truth)

In [ ]:
# Especificar hyperparámetros de las redes
in_channels_fc = ...
in_channels_cnn = ...
num_classes = ...
lr = ...
batch_size = ...
n_epochs = ...

In [ ]:
# Red 1
model_fc = ...

# Loss and optimizer
criterion = ...
optimizer = ...

...

In [ ]:
# train
def train(model, train_loader, num_epochs):
    
    for e in range(n_epochs):
        for i, (data, label) in enumerate(train_loader):
            data = data
            targets = label
            
            #Forward
            ...
            
            #Backward
            ...
            
            # Descenso del gradiente
            ...

# Evaluate
def evaluate(loader, model):
    
    model.eval()
    with torch.no_grad():
        ...

## 1.7 Comparación de Resultados

<p align="center">
  <img src="https://media2.giphy.com/media/icJA0VF7ntoEL18Jez/giphy.gif"  width="200">
</p>

Construidas las funciones de entrenamiento y evaluación, entrene a las redes que construyo y compare los resultados obtenidos con todas las redes señalando cual posee mejor rendimiento. Comente una diferencia entre las redes Fully Connected y CNN podría generar un mejor desempeño en una u otra en la tarea de clasificación de imágenes.

- [ ] Entrenar las redes.
- [ ] Evaluar las redes.
- [ ] Comentar los resultados obtenidos y visualizar si existe una diferencia significativa en el rendimiento debido a la naturaleza de la red.

In [ ]:
# Código aquí

> Comente los resultados

# Conclusión

Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/vKSR-ZakVMIAAAAC/pochitadancing-pochita.gif">
</p>